## E. Exploratory Longitudinal Health Outcome Analysis

**Description**  
This section investigates whether the level of AI implementation affects changes in hospital care quality over time.  
We take a longitudinal approach because the exact timing of AI implementation is not specified in the AHA dataset.  

This notebook includes code for:
- Assessing missing values using the 2025 3Q hospital quality dataset,
- Selecting quality metrics for longitudinal analysis,
- Performing LASSO-based variable selection for each outcome,
- Fitting linear mixed models (LMMs).

**Purpose**  
To examine whether hospital-level AI implementation is associated with changes in care quality over time.



### 1 load necessary libraries, functions, and preprocessed data 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import geopandas as gpd
from libpysal.weights import KNN, DistanceBand
from esda.moran import Moran
from spreg import ML_Error, OLS
from shapely.geometry import Point
from scipy import stats

In [ ]:
# Import functions
import sys
sys.path.append('../')
from calculate_scores import create_union_aipred_row, apply_ai_scores_to_dataframe

# Load data
AHA_master = pd.read_csv('../../data/AHA_master_external_data.csv', low_memory=False)

# Create aipred_it_union separately (your choice, works perfectly)
AHA_master['aipred_it_union'] = AHA_master.apply(create_union_aipred_row, axis=1)

# Use the apply function for all other scores
AHA_master2 = apply_ai_scores_to_dataframe(AHA_master)
AHA_IT = AHA_master2[AHA_master2['id_it'].notna()]


### 2 Data engineering  

These hospital characteristics were selected based on investigator consensus, and we used LASSO regression analysis to explore and identify additional variables that predict AI/ML implementation and reflect hospital resource levels.

- **rural_urban_type** : collected from AHA survey. categorized into {1: rural, 2: micro, 3: metro} based on the location of the hospital ('CBSATYPE')
- **system member** : hospital belonging to a corporate body that owns or manage health provider facilities or health-related subsidiaries. ('MHSMEMB')
- **delivery_system** : delivery system identified using existing theory and AHA Annual Survey data {1: Centralized Health System, 2: Centralized Physician/Insurance Health System, 3: Moderately Centralized Health System, 4: Decentralized Health System, 5: Independent Hospital System, 6/Missing: Insufficient data to determine} ('CLUSTER')
- **community_hospital** : all nonfederal, short-term general, and special hospitals whose facilities and services are available to the public {0: No, 1: Yes}('CHC')
- **subsidary_hospital** : Hospital itself operates subsidiary corporation {0: No, 1: Yes} ('SUBS')
- **frontline_hospital** : Frontline facility {0: No, 1: Yes} ('FRTLN')
- **joint_commission_accreditaion** : Accreditation by joint commision {0: No, 1: Yes} ('MAPP1')
- **center_quality** : Center for Improvement in Healthcare Quality Accreditation {0: No, 1: Yes} ('MAPP22')
- **teaching_hospital** : major teaching hospital ('MAPP8'), minor teaching hospital ('MAPP3' or 'MAPP5')
- **critical_access** critical access hospital {0: No, 1: Yes} ('MAPP18')
- **rural_referral** : rural referral center {0: No, 1: Yes} ('MAPP19')
- **ownership_type** : type of organization responsible for establishing policy concerning overall operation {government_federal, government_nonfederal, nonprofit, forprofit, other} ('CNTRL')
- **bedsize** : bed-size category, ordinal variable ('BSC')
- **medicare_ipd_percentage** : medicare inpatient days / total inpatient days. Proxy variable to reflect the proportion of medicare patient 
- **medicaid_ipd_percentage** : medicaid inpatient days / total inpatient days. Proxy variable to reflect the proportion of medicaid patients 
- **core_index** : summary measure to track the interoperability of US hospitals (https://doi.org/10.1093/jamia/ocae289)
- **friction_index** : summary measures to track the barrier or difficulty in interoperability between hospitals (https://doi.org/10.1093/jamia/ocae289)


In [ ]:
## rural_urban_type
# Continue with CBSA type and other variables
AHA_IT['rural_urban_type'] = AHA_IT['cbsatype_as'].map({
    'Rural': 1,      # Rural = 1 (lowest)
    'Micro': 2,      # Micropolitan = 2 (middle)
    'Metro': 3       # Metropolitan = 3 (highest)
})
children = [50, 51, 52, 53, 55, 56, 57, 58, 59, 90, 91]
AHA_IT['children_hospital'] = AHA_IT['serv_as'].isin(children)

## system_member
# Create new column 'system_member' based on the conditions
AHA_IT['system_member'] = AHA_IT['mhsmemb_as'].copy()
# Set to 1 where sysid_as is not null and mhsmemb_as is null
AHA_IT.loc[(AHA_IT['sysid_as'].notna()) & (AHA_IT['mhsmemb_as'].isna()), 'system_member'] = 1
# Convert all remaining null values to 0
AHA_IT['system_member'] = AHA_IT['system_member'].fillna(0).astype(int)

## AHA System Cluster Code - delivery_system
# Handle NaN values before converting to int
AHA_IT['delivery_system'] = AHA_IT['cluster_as'].fillna(0).astype(int)

## community_hospital
# Handle NaN values before converting to int
AHA_IT['community_hospital'] = AHA_IT['chc_as'].fillna(0).replace(2, 0).astype(int)

## subsidary_hospital
# Handle NaN values before converting to int
AHA_IT['subsidary_hospital'] = AHA_IT['subs_as'].fillna(0).astype(int)

## frontline_hospital
# Handle both '.' and NaN values before converting to int
AHA_IT['frontline_hospital'] = AHA_IT['frtln_as'].replace('.', 0).fillna(0).astype(int)

## joint_commission_accreditation
# Handle NaN values before converting to int
AHA_IT['joint_commission_accreditation'] = AHA_IT['mapp1_as'].fillna(0).replace(2, 0).astype(int)

## center_quality
# Handle NaN values before converting to int
AHA_IT['center_quality'] = AHA_IT['mapp22_as'].fillna(0).replace(2, 0).astype(int)

# teaching hospitals - Handle NaN values in the underlying columns
AHA_IT['teaching_hospital'] = ((AHA_IT['mapp5_as'].fillna(0) == 1) | 
                                   (AHA_IT['mapp3_as'].fillna(0) == 1) | 
                                   (AHA_IT['mapp8_as'].fillna(0) == 1)).astype(int)

AHA_IT['major_teaching_hospital'] = (AHA_IT['mapp8_as'].fillna(0) == 1).astype(int)

AHA_IT['minor_teaching_hospital'] = (((AHA_IT['mapp5_as'].fillna(0) == 1) | 
                                          (AHA_IT['mapp3_as'].fillna(0) == 1)) & 
                                         ~(AHA_IT['mapp8_as'].fillna(0) == 1)).astype(int)

# critical access hospital
AHA_IT['critical_access'] = (AHA_IT['mapp18_as'].fillna(0) == 1).astype(int)

# rural referral center 
AHA_IT['rural_referral'] = (AHA_IT['mapp19_as'].fillna(0) == 1).astype(int)

# medicare medicaid percentage - Handle division by zero and NaN values
# Replace inf and NaN with 0 for percentages
AHA_IT['medicare_ipd_percentage'] = (AHA_IT['mcripd_as'].fillna(0) / 
                                         AHA_IT['ipdtot_as'].replace(0, 1).fillna(1) * 100)
AHA_IT['medicare_ipd_percentage'] = AHA_IT['medicare_ipd_percentage'].replace([np.inf, -np.inf], 0).fillna(0)

AHA_IT['medicaid_ipd_percentage'] = (AHA_IT['mcdipd_as'].fillna(0) / 
                                         AHA_IT['ipdtot_as'].replace(0, 1).fillna(1) * 100)
AHA_IT['medicaid_ipd_percentage'] = AHA_IT['medicaid_ipd_percentage'].replace([np.inf, -np.inf], 0).fillna(0)

# bed size - Handle NaN values
AHA_IT['bedsize'] = AHA_IT['bsc_as'].fillna(0).astype(int)

# hospital ownership type - Handle NaN values before comparison
AHA_IT['nonfederal_government'] = ((AHA_IT['cntrl_as'].fillna(0) == 12) | 
                                       (AHA_IT['cntrl_as'].fillna(0) == 13) |
                                       (AHA_IT['cntrl_as'].fillna(0) == 14) | 
                                       (AHA_IT['cntrl_as'].fillna(0) == 15) | 
                                       (AHA_IT['cntrl_as'].fillna(0) == 16)).astype(int)

AHA_IT['non_profit_nongovernment'] = ((AHA_IT['cntrl_as'].fillna(0) == 21) | 
                                          (AHA_IT['cntrl_as'].fillna(0) == 23)).astype(int)

AHA_IT['for_profit'] = ((AHA_IT['cntrl_as'].fillna(0) == 31) | 
                            (AHA_IT['cntrl_as'].fillna(0) == 32) | 
                            (AHA_IT['cntrl_as'].fillna(0) == 33)).astype(int)

AHA_IT['federal_government'] = ((AHA_IT['cntrl_as'].fillna(0) == 40) | 
                                    (AHA_IT['cntrl_as'].fillna(0) == 44) | 
                                    (AHA_IT['cntrl_as'].fillna(0) == 45) | 
                                    (AHA_IT['cntrl_as'].fillna(0) == 46) | 
                                    (AHA_IT['cntrl_as'].fillna(0) == 47) | 
                                    (AHA_IT['cntrl_as'].fillna(0) == 48)).astype(int)

# Create a categorical column for hospital ownership types
def create_ownership_category(row):
    cntrl_val = row['cntrl_as'] if pd.notna(row['cntrl_as']) else 0
    if cntrl_val in [12, 13, 14, 15, 16]:
        return 'nonfederal_government'
    elif cntrl_val in [21, 23]:
        return 'non_profit_nongovernment'
    elif cntrl_val in [31, 32, 33]:
        return 'for_profit'
    elif cntrl_val in [40, 44, 45, 46, 47, 48]:
        return 'federal_government'
    else:
        return 'other'

# Create the categorical column
AHA_IT['ownership_type'] = AHA_IT.apply(create_ownership_category, axis=1)

In [ ]:
hospital_features = [
"children_hospital",
"teaching_hospital",
"nonfederal_government",
"non_profit_nongovernment",
"for_profit",
"federal_government",
"critical_access",
"rural_referral",
"medicare_ipd_percentage",
"medicaid_ipd_percentage",
"bedsize",
"delivery_system",
"community_hospital",
"subsidary_hospital",
"frontline_hospital",
"joint_commission_accreditation",
"system_member"]
coordinates_features = ["latitude_address",
"longitude_address"]
geo_features = ["rural_urban_type",
"national_adi_median",
"svi_themes_median",
"svi_theme1_median",
"svi_theme2_median",
"svi_theme3_median",
"svi_theme4_median",
"Device_Percent",
"Broadband_Percent",
"Internet_Percent",
"mean_primary_hpss",
"mean_dental_hpss",
"mean_mental_hpss",
"mean_mua_shortage",
"mean_mua_elders_shortage",
"mean_mua_infant_shortage"]
interoperability_features = ["core_index", "friction_index"]
all_covariates = hospital_features + geo_features

In [ ]:
import pandas as pd
import numpy as np
import os

#  setup
aha_df = AHA_IT.copy()
aha_df['mcrnum_as'] = aha_df['mcrnum_as'].astype(str).str.replace('.0', '')
full_hospital_list = aha_df['mcrnum_as'].unique()
print(f"Total number of hospitals in AHA: {len(full_hospital_list)}")
print(f"Sample AHA hospital IDs: {full_hospital_list[:5]}")

data_order = ['01_2022', '04_2022', '07_2022', '10_2022', 
              '01_2023', '04_2023', '07_2023', '10_2023', 
              '01_2024', '04_2024', '07_2024', '10_2024', 
              '02_2025', '04_2025']

outcome_files = {
    'General Hospital Info': "../../data/outcomes/merged_general_hospital_info.csv",
    'Death Complication': "../../data/outcomes/merged_death_complication.csv",
    'HAC Reduction': "../../data/outcomes/merged_HAC_reduction.csv",
    'Readmission': "../../data/outcomes/merged_readmission.csv",
    'Medicare Spending': "../../data/outcomes/merged_Medicare_Hospital_Spending_Per_Patient.csv",
    'Timely Care': "../../data/outcomes/merged_Timely_and_Effective_Care.csv",
    'Unplanned Visits': "../../data/outcomes/merged_Unplanned_Hospital_Visits.csv"
}



### 3 Assess missingness

In [ ]:
# Create merged_df with April 2025 data based on your missingness analysis
print("Creating merged_df with August 2025 outcome data for LASSO analysis...")

# Start with your processed AHA data (same as your analysis)
merged_df = AHA_IT.copy()
merged_df['mcrnum_as'] = merged_df['mcrnum_as'].astype(str).str.replace('.0', '')
print(f"Starting with AHA data: {merged_df.shape}")

# Use the same outcome files and time point from your analysis
recent_time = '08_2025'
outcome_files = {
    'General Hospital Info': "../../data/outcomes/merged_general_hospital_info.csv",
    'Death Complication': "../../data/outcomes/merged_death_complication.csv",
    'HAC Reduction': "../../data/outcomes/merged_HAC_reduction.csv",
    'Readmission': "../../data/outcomes/merged_readmission.csv",
    'Medicare Spending': "../../data/outcomes/merged_Medicare_Hospital_Spending_Per_Patient.csv",
    'Timely Care': "../../data/outcomes/merged_Timely_and_Effective_Care.csv",
    'Unplanned Visits': "../../data/outcomes/merged_Unplanned_Hospital_Visits.csv"
}

print(f"Merging August 2025 ({recent_time}) outcome data...")

# Track what gets merged
merge_summary = []

# Merge each outcome file - April 2025 data only
for outcome_name, file_path in outcome_files.items():
    print(f"\nProcessing {outcome_name}...")
    
    try:
        # Read the file
        df = pd.read_csv(file_path, low_memory=False)
        df = df.replace('Not Available', np.nan)
        print(f"  Loaded file: {df.shape}")
        
        # Filter to April 2025 data only
        august_df = df[df['time_point'] == recent_time].copy()
        print(f"  August 2025 data: {august_df.shape}")
        
        if len(august_df) == 0:
            print(f"  ❌ No August 2025 data found")
            continue
        
        # Prepare facility ID for merging
        august_df['Facility ID'] = august_df['Facility ID'].astype(str).str.replace('.0', '')
        
        # Check overlap with AHA hospitals
        overlap = len(set(august_df['Facility ID']) & set(merged_df['mcrnum_as']))
        print(f"  Hospital overlap: {overlap}/{len(merged_df)}")
        
        # Get columns to merge (exclude ID and time columns)
        merge_columns = [col for col in august_df.columns 
                        if col not in ['Facility ID', 'time_point']]
        
        # Add outcome-specific prefix to avoid column name conflicts
        prefix = outcome_name.replace(' ', '_') + '_'
        rename_dict = {col: prefix + col for col in merge_columns}
        august_df_prefixed = august_df.rename(columns=rename_dict)
        
        # Select columns for merging
        merge_data = august_df_prefixed[['Facility ID'] + [prefix + col for col in merge_columns]].copy()
        
        # Merge with main dataframe
        before_merge = merged_df.shape[1]
        merged_df = pd.merge(
            merged_df, 
            merge_data, 
            left_on='mcrnum_as', 
            right_on='Facility ID', 
            how='left'  # Keep all AHA hospitals
        )
        
        # Clean up duplicate Facility ID column
        merged_df = merged_df.drop(columns=['Facility ID'], errors='ignore')
        
        after_merge = merged_df.shape[1]
        columns_added = after_merge - before_merge
        
        print(f"  ✓ Added {columns_added} columns")
        print(f"  New shape: {merged_df.shape}")
        
        # Track merge statistics
        merge_summary.append({
            'outcome': outcome_name,
            'original_rows': len(august_df),
            'hospitals_overlap': overlap,
            'columns_added': columns_added,
            'merge_successful': True
        })
        
    except Exception as e:
        print(f"  ❌ Error: {str(e)}")
        merge_summary.append({
            'outcome': outcome_name,
            'merge_successful': False,
            'error': str(e)
        })


# Check data types
print(f"\nData type summary:")
print(merged_df.dtypes.value_counts())

print(f"\n✓ merged_df created successfully with August 2025 outcome data!")
print(f"Ready for LASSO analysis with shape: {merged_df.shape}")

# Optional: Save the merged dataframe for future use
merged_df.to_csv('merged_df_august_2025.csv', index=False)


In [ ]:
merged_df = pd.read_csv('merged_df_august_2025.csv')
# Simple missingness analysis of merged_df
print(f"merged_df shape: {merged_df.shape}")

# Find outcome columns (columns with prefixes from merging)
outcome_cols = [col for col in merged_df.columns 
               if any(prefix in col for prefix in ['Timely_Care_', 'Death_Complication_', 'HAC_Reduction_', 
                                                  'Readmission_', 'Medicare_Spending_', 'Unplanned_Visits_', 
                                                  'General_Hospital_Info_'])]

print(f"Found {len(outcome_cols)} outcome columns")

# Calculate missingness for each outcome column
results = []
for col in outcome_cols:
    missing_pct = (merged_df[col].isnull().sum() / len(merged_df)) * 100
    results.append({'column': col, 'missing_pct': missing_pct})

# Sort by missingness
results_df = pd.DataFrame(results).sort_values('missing_pct')

# Show results
print(f"\nColumns with < 50% missing (good for LASSO):")
good_cols = results_df[results_df['missing_pct'] < 50]
for _, row in good_cols.iterrows():
    print(f"  {row['column']}: {row['missing_pct']:.1f}% missing")

print(f"\nFound {len(good_cols)} usable columns for LASSO analysis")

In [ ]:
# Simple LASSO analysis on your April 2025 merged_df
print("Running LASSO analysis on August 2025 data...")
print(f"merged_df shape: {merged_df.shape}")

# Find outcome columns (from your merged data)
outcome_cols = [col for col in merged_df.columns 
               if any(prefix in col for prefix in ['Timely_Care_', 'Death_Complication_', 'HAC_Reduction_', 
                                                  'Readmission_', 'Medicare_Spending_', 'Unplanned_Visits_'])]

### 4 Conduct LASSO to select outcome-specific covariates 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score

# Load your merged data
merged_df = pd.read_csv('merged_df_august_2025.csv')

# Find outcome columns (from your merged data)
outcome_cols = [col for col in merged_df.columns 
               if any(prefix in col for prefix in ['Timely_Care_', 'Death_Complication_', 'HAC_Reduction_', 
                                                  'Readmission_', 'Medicare_Spending_', 'Unplanned_Visits_'])]

print(f"Found {len(outcome_cols)} outcome columns")

# Count continuous columns (numeric columns)
continuous_cols = []
for col in outcome_cols:
    # Check if the column is numeric
    if merged_df[col].dtype in ['int64', 'float64']:
        continuous_cols.append(col)
    else:
        # Try to convert to numeric to see if it's actually continuous
        try:
            pd.to_numeric(merged_df[col], errors='coerce')
            # If conversion works and we have numeric values, consider it continuous
            if merged_df[col].notna().sum() > 0:  # Has some non-null values
                continuous_cols.append(col)
        except:
            pass

print(f"Number of continuous columns: {len(continuous_cols)}")
print(f"Number of non-continuous columns: {len(outcome_cols) - len(continuous_cols)}")


In [ ]:
def lasso_covariate_table(data, outcome_columns, predictor_columns, alpha_range=None):
    """
    Run LASSO regression on multiple outcomes and return feature importance table.
    
    Parameters:
    -----------
    data : pandas DataFrame
        Input data with outcomes and predictors
    outcome_columns : list
        List of outcome column names
    predictor_columns : list
        List of predictor column names
    alpha_range : list, optional
        Range of alpha values to test. Default: np.logspace(-4, 1, 50)
    
    Returns:
    --------
    table : pandas DataFrame
        Summary table with results for each outcome
    feature_importance_dict : dict
        Dictionary with feature importance for each outcome
    """
    
    if alpha_range is None:
        alpha_range = np.logspace(-4, 1, 50)
    
    # Initialize results storage
    results = []
    feature_importance_dict = {}
    
    # Prepare predictors (handle missing values)
    X = data[predictor_columns].copy()
    
    # Fill missing values with median for continuous variables
    for col in X.columns:
        if X[col].dtype in ['float64', 'int64']:
            X[col] = X[col].fillna(X[col].median())
        else:
            # For categorical, fill with mode or 0
            X[col] = X[col].fillna(0)
    
    # Standardize predictors
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(
        scaler.fit_transform(X),
        columns=X.columns,
        index=X.index
    )
    
    print(f"Analyzing {len(outcome_columns)} outcomes...")
    
    for i, outcome in enumerate(outcome_columns):
        print(f"  {i+1}/{len(outcome_columns)}: {outcome}")
        
        try:
            # Get outcome variable
            y = data[outcome].copy()
            
            # Remove rows where outcome is missing
            valid_idx = ~y.isnull()
            if valid_idx.sum() < 10:  # Need at least 10 observations
                print(f"    Skipping {outcome}: insufficient data ({valid_idx.sum()} observations)")
                continue
            
            X_clean = X_scaled[valid_idx]
            y_clean = y[valid_idx]
            
            # Run LASSO with cross-validation
            lasso = LassoCV(
                alphas=alpha_range,
                cv=5,
                random_state=42,
                max_iter=2000,
                tol=1e-4
            )
            
            lasso.fit(X_clean, y_clean)
            
            # Get feature importance
            feature_importance = pd.Series(
                lasso.coef_,
                index=predictor_columns
            )
            
            # Count non-zero coefficients
            num_selected = (feature_importance != 0).sum()
            
            # Calculate R²
            y_pred = lasso.predict(X_clean)
            r2 = r2_score(y_clean, y_pred)
            
            # Cross-validation R²
            cv_scores = cross_val_score(lasso, X_clean, y_clean, cv=5, scoring='r2')
            cv_r2_mean = cv_scores.mean()
            cv_r2_std = cv_scores.std()
            
            # Store results
            results.append({
                'outcome': outcome,
                'alpha': lasso.alpha_,
                'num_selected': num_selected,
                'r2': r2,
                'cv_r2_mean': cv_r2_mean,
                'cv_r2_std': cv_r2_std,
                'n_observations': len(y_clean)
            })
            
            # Store feature importance
            feature_importance_dict[outcome] = feature_importance
            
            print(f"    ✓ {num_selected} features selected, CV R² = {cv_r2_mean:.3f} ± {cv_r2_std:.3f}")
            
        except Exception as e:
            print(f"    ❌ Error with {outcome}: {e}")
            continue
    
    # Create results table
    table = pd.DataFrame(results)
    
    if len(table) > 0:
        table = table.sort_values('num_selected', ascending=False)
    
    return table, feature_importance_dict

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LassoCV
from sklearn.metrics import r2_score

# Simple LASSO analysis on your August 2025 merged_df
print("Running LASSO analysis on August 2025 data...")
print(f"merged_df shape: {merged_df.shape}")

# Find outcome columns (from your merged data)
outcome_cols = [col for col in merged_df.columns 
               if any(prefix in col for prefix in ['Timely_Care_', 'Death_Complication_', 'HAC_Reduction_', 
                                                  'Readmission_', 'Medicare_Spending_', 'Unplanned_Visits_'])]

print(f"Found {len(outcome_cols)} outcome columns")

# Filter to columns with < 50% missing data
good_outcomes = []
for col in outcome_cols:
    missing_pct = (merged_df[col].isnull().sum() / len(merged_df)) * 100
    if missing_pct < 50:
        good_outcomes.append(col)

print(f"Using {len(good_outcomes)} outcomes with < 50% missing data")

# Define your predictor columns (hospital + geographic features)
predictor_cols = [
    # Hospital characteristics
    'teaching_hospital', 'children_hospital', 'nonfederal_governement', 
    'non_profit_nongovernment', 'for_profit', 'federal_government', 
    'critical_access', 'rural_referral', 'medicare_ipd_percentage', 
    'medicaid_ipd_percentage', 'bedsize', 'delivery_system', 
    'community_hospital', 'subsidary_hospital', 'frontline_hospital', 
    'joint_commission_accreditation', 'center_quality', 'system_member',
    
    # Geographic characteristics
    'rural_urban_type', 'national_adi_median', 'svi_themes_median', 
    'svi_theme1_median', 'svi_theme2_median', 'svi_theme3_median', 
    'svi_theme4_median', 'Device_Percent', 'Broadband_Percent', 
    'Internet_Percent', 'mean_primary_hpss', 'mean_dental_hpss', 
    'mean_mental_hpss', 'mean_mua_shortage', 'mean_mua_elders_shortage', 
    'mean_mua_infant_shortage']

# Filter predictors that exist and have < 75% missing
good_predictors = []
for col in predictor_cols:
    if col in merged_df.columns:
        missing_pct = (merged_df[col].isnull().sum() / len(merged_df)) * 100
        if missing_pct < 75:
            good_predictors.append(col)
    else:
        print(f"Warning: Column '{col}' not found in merged_df")

print(f"Using {len(good_predictors)} predictors with < 75% missing data")

# Run LASSO analysis
if good_outcomes and good_predictors:
    print("\nRunning LASSO...")
    try:
        table, feature_importance_dict = lasso_covariate_table(
            merged_df, 
            good_outcomes, 
            predictor_columns=good_predictors
        )
        
        print("✓ LASSO completed successfully!")
        print(f"Results: {len(table)} outcomes analyzed")
        
        # Show 5 most important features for each outcome in table format
        if len(table) > 0:
            print(f"\nTop 5 most important features for each outcome:")
            print("="*60)
            
            # Create a summary table
            summary_results = []
            
            for outcome, importance_series in feature_importance_dict.items():
                # Convert Series to DataFrame with proper columns
                importance_df = pd.DataFrame({
                    'Feature': importance_series.index,
                    'Coefficient': importance_series.values,
                    'Abs_Coefficient': np.abs(importance_series.values)
                })
                
                # Sort by absolute coefficient value (descending)
                importance_df = importance_df.sort_values('Abs_Coefficient', ascending=False)
                
                # Get top 5 features
                top_5_features = importance_df.head(5)
                
                for i, (_, row) in enumerate(top_5_features.iterrows(), 1):
                    summary_results.append({
                        'Outcome': outcome,
                        'Rank': i,
                        'Feature': row['Feature'],
                        'Coefficient': row['Coefficient'],
                        'Abs_Coefficient': row['Abs_Coefficient']
                    })
            
            # Convert to DataFrame and display
            summary_df = pd.DataFrame(summary_results)
            
            # Display table for each outcome
            for outcome in summary_df['Outcome'].unique():
                outcome_data = summary_df[summary_df['Outcome'] == outcome]
                print(f"\n{outcome}:")
                print(outcome_data[['Rank', 'Feature', 'Coefficient']].to_string(index=False))
            
            # Save the complete results table
            summary_df.to_csv('lasso_top5_features_April2025.csv', index=False)
            print(f"\n✓ Saved complete results to 'lasso_top5_features_April2025.csv'")
            
            # Overall summary
            print(f"\n" + "="*60)
            print(f"OVERALL SUMMARY")
            print(f"="*60)
            print(f"Total outcomes analyzed: {len(feature_importance_dict)}")
            print(f"Average features selected per outcome: {table['num_selected'].mean():.1f}")
            
            # Most frequently selected features across all outcomes
            all_features = summary_df['Feature'].value_counts()
            if len(all_features) > 0:
                print(f"\nMost frequently selected features (top 10):")
                for feature, count in all_features.head(10).items():
                    print(f"  {feature}: selected in {count} outcomes")
        
    except Exception as e:
        print(f"❌ LASSO failed: {e}")
        import traceback
        traceback.print_exc()
        
else:
    print("❌ No suitable outcomes or predictors found")
    print(f"Outcomes: {len(good_outcomes)}, Predictors: {len(good_predictors)}")

In [ ]:
# Initialize summary_df as None at the beginning
summary_df = None
top5_features_df = None  # New DataFrame for the format you want

# Show 5 most important features for each outcome in table format
if len(table) > 0:
    
    summary_results = []
    top5_features_list = []
    
    for outcome, importance_df in feature_importance_dict.items():
        try:
            if isinstance(importance_df, pd.DataFrame):
                if len(importance_df) > 0:
                    # Get the first column as feature names and second column as coefficients
                    features = importance_df.iloc[:, 0].tolist()  # First column
                    coefficients = importance_df.iloc[:, 1].tolist()  # Second column
                    abs_coefficients = importance_df.iloc[:, 2].tolist()  # Third column
                    
                    # Create a proper DataFrame for processing
                    processed_df = pd.DataFrame({
                        'Feature': features,
                        'Coefficient': coefficients,
                        'Abs_Coefficient': abs_coefficients
                    })
                    
                    # Sort by absolute coefficient value (descending)
                    processed_df = processed_df.sort_values('Abs_Coefficient', ascending=False)
                    
                    # Get top 5 features
                    top_5_features = processed_df.head(5)
                    
                    # Store the top 5 features for this outcome
                    outcome_features = []
                    for i, (_, row) in enumerate(top_5_features.iterrows(), 1):
                        outcome_features.append(row['Feature'])
                        summary_results.append({
                            'Outcome': outcome,
                            'Rank': i,
                            'Feature': row['Feature'],
                            'Coefficient': row['Coefficient'],
                            'Abs_Coefficient': row['Abs_Coefficient']
                        })
                    
                    # Pad with None if we have fewer than 5 features
                    while len(outcome_features) < 5:
                        outcome_features.append(None)
                    
                    # Add to the top5_features_list
                    top5_features_list.append({
                        'Outcome': outcome,
                        'Feature_1': outcome_features[0],
                        'Feature_2': outcome_features[1],
                        'Feature_3': outcome_features[2],
                        'Feature_4': outcome_features[3],
                        'Feature_5': outcome_features[4]
                    })
                    
            
                else:
                    print(f"Warning: Empty DataFrame for outcome '{outcome}'")
                    
            else:
                # Handle the case where it's a Series (original expected format)
                importance_values = pd.to_numeric(importance_df.values, errors='coerce')
                valid_indices = ~pd.isna(importance_values)
                
                if valid_indices.any():
                    processed_df = pd.DataFrame({
                        'Feature': importance_df.index[valid_indices],
                        'Coefficient': importance_values[valid_indices],
                        'Abs_Coefficient': np.abs(importance_values[valid_indices])
                    })
                    
                    processed_df = processed_df.sort_values('Abs_Coefficient', ascending=False)
                    top_5_features = processed_df.head(5)
                    
                    outcome_features = []
                    for i, (_, row) in enumerate(top_5_features.iterrows(), 1):
                        outcome_features.append(row['Feature'])
                        summary_results.append({
                            'Outcome': outcome,
                            'Rank': i,
                            'Feature': row['Feature'],
                            'Coefficient': row['Coefficient'],
                            'Abs_Coefficient': row['Abs_Coefficient']
                        })
                    
                    while len(outcome_features) < 5:
                        outcome_features.append(None)
                    
                    top5_features_list.append({
                        'Outcome': outcome,
                        'Feature_1': outcome_features[0],
                        'Feature_2': outcome_features[1],
                        'Feature_3': outcome_features[2],
                        'Feature_4': outcome_features[3],
                        'Feature_5': outcome_features[4]
                    })
                    
                    print(f"✓ Successfully processed {outcome} with {len(outcome_features)} features")
                else:
                    print(f"Warning: No valid numeric values found for outcome '{outcome}'")
                
        except Exception as e:
            print(f"Error processing outcome '{outcome}': {e}")
            print(f"Importance data type: {type(importance_df)}")
            if hasattr(importance_df, 'values'):
                print(f"Importance data values (first 5): {importance_df.values[:5]}")
            continue
    
    # Convert to DataFrames and display
    if summary_results:
        summary_df = pd.DataFrame(summary_results)
        top5_features_df = pd.DataFrame(top5_features_list)
        
        
        # Display detailed table for each outcome
        for outcome in summary_df['Outcome'].unique():
            outcome_data = summary_df[summary_df['Outcome'] == outcome]
            print(f"\n{outcome}:")
            print(outcome_data[['Rank', 'Feature', 'Coefficient']].to_string(index=False))
        
        # Overall summary
        print(f"\n" + "="*60)
        print(f"OVERALL SUMMARY")
        print(f"="*60)
        print(f"Total outcomes analyzed: {len(feature_importance_dict)}")
        print(f"Average features selected per outcome: {table['num_selected'].mean():.1f}")
        
        # Most frequently selected features across all outcomes
        all_features = summary_df['Feature'].value_counts()
        if len(all_features) > 0:
            print(f"\nMost frequently selected features (top 10):")
            for feature, count in all_features.head(10).items():
                print(f"  {feature}: selected in {count} outcomes")
    else:
        print("No valid results to display")

### 5 conduct longitudinal analysis 

In [ ]:
ai_ml = ['ai_base_score_imputed']
hospital_resource = ['delivery_system', 'bedsize', 'system_member']

In [ ]:
AHA_master3 = AHA_IT[ai_ml + all_covariates + ['id_it', 'mcrnum_as']]
AHA_master3['mcrnum_as'] = AHA_master3['mcrnum_as'].astype(str).str.zfill(6)
for col in ai_ml+hospital_resource:
    AHA_master3[col] = AHA_master3[col].astype(float).fillna(0)
AHA_master3['mcrnum_as'] = pd.to_numeric(AHA_master3['mcrnum_as'], errors='coerce')
AHA_master3['mcrnum_as'] = AHA_master3['mcrnum_as'].fillna(-1).astype(int)
AHA_master3['mcrnum_as'] = AHA_master3['mcrnum_as'].apply(lambda x: str(x).zfill(6) if x != -1 else np.nan)

In [ ]:
# Fix the get_sort_index function to properly return a value
def get_sort_index(tp):
    try:
        if pd.isna(tp) or tp == 'nan':
            return np.nan
        month, year = tp.split('_')
        return f"{year}_{month.zfill(2)}"  # Returns "2022_01" format
    except:
        return np.nan
# First, let's convert the time points to a numeric format
# Assuming time_point2 is in format 'YYYY_MM'
def convert_time_to_numeric(time_str):
    year, month = time_str.split('_')
    return float(year) + (float(month) - 1) / 12  # This will give us years as decimal

In [ ]:
def calculate_adjusted_slopes_by_ai_level(df, outcome_column, additional_covariates=None):
    """
    Fit a mixed-effects model for outcome over time by AI level,
    adjusting for any selected covariates, and compute slopes per AI group.
    Uses robust variance estimation for more reliable inference.

    Parameters:
    - df: pandas.DataFrame with required columns
    - outcome_column: name of the outcome variable
    - additional_covariates: list of column names to include as fixed effects

    Returns:
    - result: fitted mixedlm result object
    - slope_df: pandas.DataFrame with estimated slopes per AI level
    - results_table: results with robust standard errors
    """
    df = df.copy()
    df[outcome_column] = pd.to_numeric(df[outcome_column], errors='coerce')
    df['time_point2'] = df['time_point'].apply(get_sort_index)
    cols = [outcome_column, 'time_point2', 'ai_base_score_imputed', 'id_it']
    if additional_covariates:
        cols += additional_covariates
    model_df = df[cols].dropna()
    
    # Numeric time and categorical AI
    model_df['time_point2_numeric'] = model_df['time_point2'].apply(convert_time_to_numeric)
    model_df['ai_cat'] = model_df['ai_base_score_imputed'].astype('category')
    
    # Build formula string
    base_terms = ['time_point2_numeric', 'ai_cat', 'time_point2_numeric:ai_cat']
    cov_terms = additional_covariates or []
    formula = f"{outcome_column} ~ " + " + ".join(base_terms + cov_terms)
    
    # Fit mixed-effects model
    model = smf.mixedlm(formula, model_df, groups='id_it')
    result = model.fit(reml=False)
    
    # Extract slopes for each AI category
    coefs = result.params
    slopes = {
        "AI 0 (reference)": coefs["time_point2_numeric"],
        "AI 1": coefs["time_point2_numeric"] + coefs.get("time_point2_numeric:ai_cat[T.1.0]", 0),
        "AI 2": coefs["time_point2_numeric"] + coefs.get("time_point2_numeric:ai_cat[T.2.0]", 0)
    }
    
    # ROBUST VARIANCE ESTIMATION
    # Use HC1 (Huber-White) robust standard errors
    try:
        # Get robust covariance matrix
        robust_cov = result.cov_params()
        
        # Calculate robust standard errors
        robust_se = np.sqrt(np.diag(robust_cov))
        
        # Calculate robust t-statistics
        robust_t_stats = result.params / robust_se
        
        # Calculate robust p-values (two-tailed)
        from scipy import stats
        robust_p_values = 2 * (1 - stats.t.cdf(np.abs(robust_t_stats), df=len(model_df) - len(result.params)))
        
        # Calculate robust confidence intervals
        robust_ci_lower = result.params - 1.96 * robust_se
        robust_ci_upper = result.params + 1.96 * robust_se
        
        print("✓ Using robust variance estimation (Huber-White)")
        
    except Exception as e:
        print(f"⚠️ Robust variance estimation failed, using model-based SE: {e}")
        robust_se = result.bse
        robust_t_stats = result.params / robust_se
        robust_p_values = result.pvalues
        robust_ci_lower = result.conf_int()[0]
        robust_ci_upper = result.conf_int()[1]
    
    # Create detailed results table with robust SEs
    results_table = pd.DataFrame({
        'Coefficient': result.params,
        'Robust Std Error': robust_se,
        'Robust t-stat': robust_t_stats,
        'Robust P-value': robust_p_values,
        'Robust Lower CI': robust_ci_lower,
        'Robust Upper CI': robust_ci_upper
    }).round(5)
    
    # Create slope dataframe with more information
    slope_df = pd.DataFrame.from_dict(slopes, orient='index', columns=['Estimated Slope'])
    slope_df.index.name = "AI Level"
    slope_df = slope_df.round(7)
    
    # Display results
    print("\nSelected covariates:", additional_covariates)
    print("\nEstimated Slopes by AI Level:")
    print(slope_df)
    
    print("\nDetailed Model Results (with Robust Standard Errors):")
    print(results_table)
    
    print("\nModel Summary:")
    print(result.summary())
    
    # Extract what you need with robust SEs:
    print("\nFixed Effects Results (Robust Standard Errors):")
    print(f"{'Parameter':<30} {'Coeff':<8} {'Robust SE':<10} {'t-stat':<8} {'P-value':<10} {'95% CI':<20}")
    print("-" * 90)
    
    # FUse enumerate to get indices that align with the arrays
    for i, param in enumerate(result.params.index):
        coeff = result.params[param]
        se = robust_se[i]  # Use index i instead of param name
        t_stat = robust_t_stats[i]  # Use index i instead of param name
        p_val = robust_p_values[i]  # Use index i instead of param name
        ci_lower = robust_ci_lower[i]  # Use index i instead of param name
        ci_upper = robust_ci_upper[i]  # Use index i instead of param name
        
        print(f"{param:<30} {coeff:<8.3f} {se:<10.3f} {t_stat:<8.2f} {p_val:<10.6f} [{ci_lower:.6f}, {ci_upper:.6f}]")

    # Reset display options
    pd.reset_option('display.float_format')
    residuals = result.resid

    # Shapiro-Wilk test (use on sample if N > 5000)
    shapiro_stat, shapiro_p = stats.shapiro(residuals.dropna())
    print(f"\nShapiro-Wilk test: W = {shapiro_stat:.4f}, p = {shapiro_p:.4f}")
    
    # Q-Q plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    stats.probplot(residuals.dropna(), dist="norm", plot=ax1)
    ax1.set_title("Q-Q Plot of Residuals")

    # Histogram of residuals
    ax2.hist(residuals.dropna(), bins=50, density=True, alpha=0.7)
    ax2.set_title("Distribution of Residuals")
    plt.tight_layout()
    plt.show()

    # Plot residuals vs fitted values
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=result.fittedvalues, y=residuals, alpha=0.5)
    plt.title("Residuals vs Fitted Values")
    plt.xlabel("Fitted Values")
    plt.ylabel("Residuals")
    plt.axhline(y=0, color='red', linestyle='--', alpha=0.5)
    plt.show()

    # Breusch-Pagan test for heteroscedasticity
    try:
        from statsmodels.stats.diagnostic import het_breuschpagan
        bp_stat, bp_p, f_stat, f_p = het_breuschpagan(residuals.dropna(), 
                                                  result.model.exog[~np.isnan(residuals)])
        print(f"Breusch-Pagan test: LM = {bp_stat:.4f}, p = {bp_p:.4f}")
    except Exception as e:
        print(f"Breusch-Pagan test failed: {e}")

    return result, slope_df, results_table

In [ ]:
def visualize_adjusted_hospital_quality_over_time(df, outcome_column, additional_covariates):
    # Set publication-quality settings
    plt.style.use('seaborn-v0_8-whitegrid')
    plt.rcParams.update({
        'font.family': 'Helvetica',
        'font.size': 12,
        'axes.labelsize': 12,
        'axes.titlesize': 14,
        'xtick.labelsize': 10,
        'ytick.labelsize': 10,
        'legend.fontsize': 10,
        'figure.dpi': 300,
        'savefig.dpi': 300,
        'pdf.fonttype': 42,
        'ps.fonttype': 42,
        'axes.linewidth': 1,
        'axes.grid': True,
        'grid.alpha': 0.3,
        'grid.linestyle': '--',
        'grid.linewidth': 0.5
    })

    # Data preparation
    print("Data preparation and validation:")
    print(f"Total rows in dataset: {len(df)}")
    
    # Convert outcome to numeric and handle missing values
    df[outcome_column] = pd.to_numeric(df[outcome_column], errors='coerce')
    df['time_point'] = df['time_point'].astype(str).replace('nan', np.nan)
    
    # Apply the function to transform date format
    df['time_point2'] = df['time_point'].apply(get_sort_index)
    
    # Create model dataframe and handle missing values
    model_df = df[[outcome_column, 'time_point2', 'ai_base_score_imputed', 'id_it'] + additional_covariates].dropna()
    print(f"Rows after dropping missing values: {len(model_df)}")
    
    # Convert ai_base_score to category
    model_df['ai_cat'] = model_df['ai_base_score_imputed'].astype('category')
    
    # Create the formula string with all covariates
    formula = f"Q('{outcome_column}') ~ time_point2 + ai_cat + time_point2:ai_cat + " + " + ".join(additional_covariates)
    
    # Fit model
    model = smf.mixedlm(formula, model_df, groups='id_it').fit(reml=False)
    
    # Get unique time points from the data
    unique_time_points = sorted(model_df['time_point2'].unique())
    
    # Create prediction dataframe using only time points that exist in the data
    pred_rows = []
    for time_point in unique_time_points:
        for ai in model_df['ai_cat'].unique():
            # Use median values for additional covariates
            row = {'time_point2': time_point, 'ai_cat': ai}
            for cov in additional_covariates:
                row[cov] = model_df[cov].median()
            pred_rows.append(row)
    
    pred_df = pd.DataFrame(pred_rows)
    
    # Add the outcome column (required by statsmodels)
    pred_df[outcome_column] = 0  # This value doesn't matter for prediction
    
    # Get predictions
    pred_df['predicted'] = model.predict(pred_df)
    
    # Create mapping for x-axis labels
    quarter_labels = {
        '2022_01': 'Q1 2022',
        '2022_04': 'Q2 2022',
        '2022_07': 'Q3 2022',
        '2022_10': 'Q4 2022',
        '2023_01': 'Q1 2023',
        '2023_04': 'Q2 2023',
        '2023_07': 'Q3 2023',
        '2023_10': 'Q4 2023',
        '2024_01': 'Q1 2024',
        '2024_04': 'Q2 2024',
        '2024_07': 'Q3 2024',
        '2024_10': 'Q4 2024',
        '2025_02': 'Q1 2025',
        '2025_04': 'Q2 2025',
        '2025_08': 'Q3 2025'
    }
    
    # Add numeric indices for proper sorting
    pred_df['x_idx'] = pred_df['time_point2'].map(lambda x: list(unique_time_points).index(x))
    
    # Create figure
    fig, ax = plt.subplots(figsize=(10, 6), dpi=300)
    
    # Define custom color palette
    colors = {
        0: '#808080',  # Gray for AI 0
        1: '#2ca02c',  # Green for AI 1
        2: '#1f77b4'   # Blue for AI 2
    }
    
    # Plot for each AI level
    for ai in sorted(pred_df['ai_cat'].unique()):
        subset = pred_df[pred_df['ai_cat'] == ai].sort_values('x_idx')
        
        # Plot lines connecting available data points
        ax.plot(subset['x_idx'], subset['predicted'], 
                label=f'AI {int(ai)}', 
                marker='o',
                color=colors[int(ai)],
                linewidth=2,
                markersize=8,
                markeredgecolor='white',
                markeredgewidth=1)
    
    # Set x-ticks
    x_ticks = range(len(unique_time_points))
    x_labels = [quarter_labels.get(tp, tp) for tp in unique_time_points]
    
    ax.set_xticks(x_ticks)
    ax.set_xticklabels(x_labels, rotation=45, ha='right')
    
    # Set axis limits
    ax.set_xlim(-0.5, len(unique_time_points) - 0.5)
    
    # Calculate appropriate y-axis limits
    y_min = pred_df['predicted'].min() - 0.05 * (pred_df['predicted'].max() - pred_df['predicted'].min())
    y_max = pred_df['predicted'].max() + 0.05 * (pred_df['predicted'].max() - pred_df['predicted'].min())
    ax.set_ylim(y_min, y_max)
    
    # Customize the plot
    ax.set_xlabel("Quarter", fontweight='bold', labelpad=10)
    ax.set_ylabel("Adjusted Quality Metric", fontweight='bold', labelpad=10)
    
    # Add title and subtitle
    ax.set_title(f"Adjusted {outcome_column} Over Time by AI Level", 
                 fontweight='bold', pad=20)

    
    # Add legend
    legend = ax.legend(title="AI Level", 
                      title_fontsize=12,
                      frameon=True,
                      framealpha=0.95,
                      edgecolor='black',
                      loc='best')
    
    # Add grid
    ax.grid(True, linestyle='--', alpha=0.3)
    
    # Remove top and right spines
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    
    # Adjust layout
    plt.tight_layout()
    
    # Print model summary and coefficient values
    print("\nModel Summary Statistics:")
    print(f"Number of observations: {len(model_df)}")
    print(f"Number of hospitals: {model_df['id_it'].nunique()}")
    print("\nFixed Effects:")
    print(model.summary().tables[1])
    
    # Print the actual coefficient values for additional covariates
    print("\nCoefficients for additional covariates:")
    for cov in additional_covariates:
        if cov in model.params.index:
            print(f"{cov}: {model.params[cov]:.4f}")
    
    plt.show()

In [ ]:
general_hospital = pd.read_csv("../../data/outcomes/merged_general_hospital_info.csv", low_memory=False)
general_hospital = general_hospital.replace('Not Available', np.nan)
AHA_master3 = AHA_IT[ai_ml + all_covariates + ['id_it', 'mcrnum_as']]
AHA_master3['mcrnum_as'] = AHA_master3['mcrnum_as'].astype(str).str.zfill(6)
for col in ai_ml+hospital_resource:
    AHA_master3[col] = AHA_master3[col].astype(float).fillna(0)


In [ ]:
death_complication = pd.read_csv("../../data/outcomes/merged_death_complication.csv", low_memory=False)
death_complication = death_complication.replace('Not Available', np.nan)
death_complication_AHA = AHA_master3.merge(death_complication, left_on = 'mcrnum_as', right_on = 'Facility ID', how = 'left') 

In [ ]:
# example use case 
calculate_adjusted_slopes_by_ai_level(death_complication_AHA, 'MORT_30_PN', additional_covariates= ['delivery_system',
  'rural_urban_type',
  'non_profit_nongovernment',
  'mean_primary_hpss',
  'national_adi_median',
  'bedsize',
  'system_member'])